In [1]:
import pandas as pd
import numpy as np
import os

## 1. 멀티 인덱스

- 인덱스로 [그룹1의 123, 그룹2의 123] 처럼 2겹으로 만들어야 할 때 사용<br><br>

- 행의 이름이 두겹으로 들어감<br><br>

- 칼럼은 일반적인 칼럼

In [2]:
my_header = ['a','b','c']

In [3]:
my_index_out = ['G1']*3 + ['G2']*3    #G1을 3번, G2를 3번 갖는 리스트 만듬  (G1G1G1, G2G2G2)
my_index_in = [1,2,3]*2               

In [4]:
my_index_zipped = list(zip(my_index_out, my_index_in))   #G1 123리스트, G2 123리스트 맞물려 튜플 생성
my_index_zipped

[('G1', 1), ('G1', 2), ('G1', 3), ('G2', 1), ('G2', 2), ('G2', 3)]

In [5]:
my_index = pd.MultiIndex.from_tuples(my_index_zipped)    #zipped된 걸 멀티인덱스에 넣어주면 됨
df = pd.DataFrame(data=np.random.randn(6,3),index=my_index,columns=my_header)

In [6]:
df

a         b         c
G1 1  0.668442 -0.351210  0.221961
   2 -0.062280  0.080892  0.282558
   3  1.223447  0.326909 -0.360229
G2 1 -0.037508  0.278647 -0.354152
   2  0.292700  1.581152  0.131434
   3 -0.039159 -0.103121  0.462127

### 슬라이싱:

In [7]:
df.loc['G1']   #G1 행 가져옴

,a,b,c
1,0.668442,-0.351210,0.221961
2,-0.062280,0.080892,0.282558
3,1.223447,0.326909,-0.360229


In [8]:
df.loc['G1'].loc[1]   #G1의 1행

a    0.668442
b   -0.351210
c    0.221961
Name: 1, dtype: float64

In [9]:
df.loc['G1'].loc[1,'b']  #G1그룹의 1행의 b 

-0.35121012722516215

## 2. groupby

- 그룹별 연산<br><br>
- 남자의 신장평균, 여자의 신장평균 따로 따로 구하는 것이 아님<br><br>
- 성별 신장평균처럼 동시에 구할 때 사용<br><br>
- groupby로 나오는 결과는 시리즈

In [10]:
os.chdir(r"C:\Users\Gram\Desktop\아시아경제 수업자료\01 Python 분석 기초 - 실습\data")
df = pd.read_csv('data_studentlist_en.csv',header='infer',encoding = 'latin1')

In [11]:
df.head(3)     #몇개만 뽑아서 표 확인

,name,gender,age,grade,absence,bloodtype,height,weight
0,Jared Diamond,M,23,3,Y,O,165.3,68.2
1,Sarah O'Donnel,F,22,2,N,AB,170.1,53.0
2,Brian Martin,M,24,4,N,B,175.0,80.1


In [12]:
df.groupby('gender').mean()   #모든 변수 각각의 평균

,age,grade,height,weight
gender,,,,
F,21.857143,1.714286,166.642857,50.442857
M,22.700000,2.700000,172.410000,68.500000


In [13]:
df.groupby('gender')['height'].mean()    #성별 신장 평균

gender
F    166.642857
M    172.410000
Name: height, dtype: float64

In [14]:
df.groupby('gender')[['height','weight']].mean()    #성별 신장,체중 평균

,height,weight
gender,,
F,166.642857,50.442857
M,172.410000,68.500000


In [15]:
df.groupby('gender')[['grade','age']].std()    #성별 성적,나이 표준편차

,grade,age
gender,,
F,0.755929,1.069045
M,1.159502,1.159502


In [16]:
df.groupby('gender')['height'].describe()   #성별 신장에 대한 통계요약

,count,mean,std,min,25%,50%,75%,max
gender,,,,,,,,
F,7.0,166.642857,8.487414,155.2,160.100,168.0,173.1,176.9
M,10.0,172.410000,6.804647,162.2,167.475,172.1,177.9,182.1


In [17]:
sr = df.groupby(['gender','bloodtype'])['height'].mean()     # 멀티 인덱싱된 시리즈
sr               #성별과 혈액 두 그룹화(명목형 변수만 들어가야 함)

gender  bloodtype
F       A            172.450000
        AB           170.100000
        B            158.200000
        O            164.433333
M       A            165.700000
        AB           181.050000
        B            174.550000
        O            166.200000
Name: height, dtype: float64

In [18]:
sr.loc['F']    #그룹 F의 데이터

bloodtype
A     172.450000
AB    170.100000
B     158.200000
O     164.433333
Name: height, dtype: float64

In [19]:
sr.loc['F'].loc['A']    #그룹F에서 A혈액형

172.45

## 3. apply 메서드

- 데이터프레임에 적용되는 메서드가 아닌 시리즈 하나하나에 적용되는 메서드<br><br>

- 람다함수를 인자로 갖음
- 람다함수는 한번 쓰고 버림

In [20]:
df['height'].apply(lambda x: x/100)   

0     1.653
1     1.701
2     1.750
3     1.821
4     1.680
5     1.620
6     1.552
7     1.769
8     1.785
9     1.761
10    1.671
11    1.800
12    1.622
13    1.761
14    1.582
15    1.686
16    1.692
Name: height, dtype: float64

## 4. 정렬

In [21]:
df.sort_values(by='bloodtype')   #혈액형 기준으로 데이터프레임 소팅

,name,gender,age,grade,absence,bloodtype,height,weight
16,Andrew Daley,M,21,1,N,A,169.2,62.2
4,Clara Rodriquez,F,20,1,Y,A,168.0,49.5
12,Eddy Johnson,M,21,1,N,A,162.2,55.3
7,Margareth Jones,F,23,1,N,A,176.9,55.0
1,Sarah O'Donnel,F,22,2,N,AB,170.1,53.0
3,David Hassel,M,23,3,N,AB,182.1,85.7
11,John Matsuda,M,22,2,N,AB,180.0,75.8
14,Linda Carter,F,22,2,N,B,158.2,45.2
9,Jake Timmerman,M,22,2,N,B,176.1,61.3
8,John Bertsch,M,23,3,N,B,178.5,64.2


In [22]:
df.sort_values(by='bloodtype', ascending=False)    #혈액형 기준으로 소팅, 내림차순으로 정렬

,name,gender,age,grade,absence,bloodtype,height,weight
0,Jared Diamond,M,23,3,Y,O,165.3,68.2
13,Rebecah Anderson,F,23,3,N,O,176.1,53.1
5,Jennifer Lorentz,F,21,2,N,O,162.0,52.0
6,Susan Clark,F,22,1,N,O,155.2,45.3
10,Joshua Connor,M,24,4,Y,O,167.1,62.0
9,Jake Timmerman,M,22,2,N,B,176.1,61.3
15,Richard Swayze,M,24,4,Y,B,168.6,70.2
14,Linda Carter,F,22,2,N,B,158.2,45.2
8,John Bertsch,M,23,3,N,B,178.5,64.2
2,Brian Martin,M,24,4,N,B,175.0,80.1


In [23]:
df.sort_values(by=['bloodtype','gender'])     #혈액형 기준하고, 성별기준

,name,gender,age,grade,absence,bloodtype,height,weight
4,Clara Rodriquez,F,20,1,Y,A,168.0,49.5
7,Margareth Jones,F,23,1,N,A,176.9,55.0
12,Eddy Johnson,M,21,1,N,A,162.2,55.3
16,Andrew Daley,M,21,1,N,A,169.2,62.2
1,Sarah O'Donnel,F,22,2,N,AB,170.1,53.0
3,David Hassel,M,23,3,N,AB,182.1,85.7
11,John Matsuda,M,22,2,N,AB,180.0,75.8
14,Linda Carter,F,22,2,N,B,158.2,45.2
2,Brian Martin,M,24,4,N,B,175.0,80.1
8,John Bertsch,M,23,3,N,B,178.5,64.2


## 5. 명목형 변수 요약:

- 명목형변수로 도수분포표 만들때 사용

In [24]:
df['bloodtype'].unique()     #유형 보여줌     #a,b,ab,o

array(['O', 'AB', 'B', 'A'], dtype=object)

In [25]:
df['bloodtype'].nunique()    #종류 개수

4

In [26]:
#도수분포표
df['bloodtype'].value_counts()

B     5
O     5
A     4
AB    3
Name: bloodtype, dtype: int64

In [27]:
#도수분포표
df['gender'].value_counts()

M    10
F     7
Name: gender, dtype: int64

## 6. 피보팅

- 유형으로 그룹을 쪼개 통계치 구할 때 사용<br><br>
- group by로 나오는 결과는 시리즈
- 피봇으로 나오는 결과는 데이터프레임<br><br>
- 피보팅에는 기본값이 aggregate로 설정되어 있음<br><br>
- 피보팅은 aggregate,apply를 한번에 계산함

In [28]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                          "bar", "bar", "bar", "bar"],
                    "B": ["one", "one", "one", "two", "two",
                          "one", "one", "two", "two"],
                    "C": ["small", "large", "large", "small",
                          "small", "large", "small", "small",
                          "large"],
                    "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                    "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


### 6.1 A,B의 값으로 인덱스, C의 값으로 컬럼, 실제 셀에 들어가는 값은 E의 평균

In [29]:
dfr = pd.pivot_table(df, index=['A','B'], columns='C', values='E') #values=E 는 실제값을 지정
                         #A,B의 칼럼으로 인덱스 삼음               #그룹별 뭘 계산하고 싶은가에 달라짐
                         #A,B,C중 아무거나 상관없음                #성적 계산하고 싶다면 성적칼럼을 values에 넣음
dfr

#a,b,c만 레이블, e는 값

C        large  small
A   B                
bar one    6.0    8.0
    two    9.0    9.0
foo one    4.5    2.0
    two    NaN    5.5

In [30]:
dfr.columns

Index(['large', 'small'], dtype='object', name='C')

In [31]:
dfr.index       # 행의 인덱스가 멀티인덱스 

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('foo', 'one'),
            ('foo', 'two')],
           names=['A', 'B'])

In [32]:
pd.pivot_table(df, index=['A','B'], columns='C', values='E', fill_value=0) #유형에 해당하는 행이 없으면 na로 나오므로 0으로 메꿔줌
                                                                           #계산을 위해 0으로

C        large  small
A   B                
bar one    6.0    8.0
    two    9.0    9.0
foo one    4.5    2.0
    two    0.0    5.5

### 6.2 A,B의 값으로 인덱스, C의 값으로 컬럼, 실제 셀에 들어가는 값은 E의 중앙값

In [33]:
pd.pivot_table(df, index=['A','B'], columns='C', values='E', aggfunc=np.median,fill_value=0) 
                                                             #디폴트가 평균이므로 중앙값 계산하고플때

C        large  small
A   B                
bar one    6.0    8.0
    two    9.0    9.0
foo one    4.5    2.0
    two    0.0    5.5

### 6.3 C의 값으로 인덱스, A, B의 값으로 컬럼, 실제 셀에 들어가는 값은 E의 평균

In [34]:
dfr = pd.pivot_table(df, index='C', columns=['A','B'], values='E')
dfr

A      bar       foo     
B      one  two  one  two
C                        
large  6.0  9.0  4.5  NaN
small  8.0  9.0  2.0  5.5

In [35]:
dfr.index

Index(['large', 'small'], dtype='object', name='C')

In [36]:
dfr.columns          # 이제는 컬럼이 멀티인덱스 

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('foo', 'one'),
            ('foo', 'two')],
           names=['A', 'B'])

### 6.4 D와 E의 그룹평균

In [37]:
pd.pivot_table(df, index=['A','B'], values=['D','E'], aggfunc=np.mean) #a,b로 소그룹하고 그룹별 d의 평균, e의 평균
                                     
#레이블을 제공하는 변수 a,b (인덱스)
#계산을 제공하는 수치형 변수 d,e (벨류)

D         E
A   B                      
bar one  4.500000  7.000000
    two  6.500000  9.000000
foo one  1.666667  3.666667
    two  3.000000  5.500000

In [38]:
# 위 결과와 비교.
df.groupby(['A','B'])[['D','E']].mean()

D         E
A   B                      
bar one  4.500000  7.000000
    two  6.500000  9.000000
foo one  1.666667  3.666667
    two  3.000000  5.500000

### 6.5 D, E를 다른 방법으로 집계함

In [39]:
pd.pivot_table(df, index=['A','B'], values=['D','E'], aggfunc={'D':np.mean,'E':np.median})  #D는 평균, E는 중앙값
                                                               

D    E
A   B                 
bar one  4.500000  7.0
    two  6.500000  9.0
foo one  1.666667  4.0
    two  3.000000  5.5